<a href="https://colab.research.google.com/github/KimGriezmann/DeepLearning_BeTheLegend/blob/master/%EC%B5%9C%EA%B7%BC%205%EC%9D%BC%20%EA%B2%BD%EA%B8%B0%EC%9D%98%20%ED%83%80%EC%9C%A8%20%EA%B5%AC%ED%95%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import tqdm

In [2]:
data = pd.read_excel("/content/날짜별 타자 데이터 통합 (1).xlsx")
hitter = pd.read_excel("/content/90명 타자 명단.xlsx")

In [3]:
data = pd.DataFrame(data)
hitter = pd.DataFrame(hitter)

In [4]:
print(data.head(), len(data))

     day name  vs  타순  P  ab  hit  ...  SSG   KT        KW  KIA   HH   SS   NC
0  44289  강민호  키움   6  C   3    0  ...  0.0  0.0  0.000000  0.0  0.0  0.0  0.0
1  44290  강민호  키움   7  C   4    2  ...  0.0  0.0  0.285714  0.0  0.0  0.0  0.0
2  44292  강민호  두산   9  C   1    0  ...  0.0  0.0  0.000000  0.0  0.0  0.0  0.0
3  44293  강민호  두산   6  C   4    1  ...  0.0  0.0  0.000000  0.0  0.0  0.0  0.0
4  44294  강민호  두산   7  C   3    2  ...  0.0  0.0  0.000000  0.0  0.0  0.0  0.0

[5 rows x 25 columns] 5296


In [5]:
print(hitter.head(), len(hitter))

  Team batter
0   롯데    전준우
1   롯데    손아섭
2   롯데     정훈
3   롯데    마차도
4   롯데    한동희 90


In [6]:
data.loc[:,['day','name','ab','hit']].dtypes

day      int64
name    object
ab       int64
hit      int64
dtype: object

In [7]:
#
# 선수마다 최근 5일의 타율을 구해야함. 
# 현재 day에서 -5부터 -1까지의 sum(hit)/sum(ab) 값을 구하면 된다

# hitter_data : 90명 타자 data
# record_data : 날짜별 타자 기록
# recent_day : 기본(최근 5일), 설정 가능
def make_recent_days(hitter_data, record_data, recent_day=5):

  # 최근 5일 경기 타율 저장할 공간 생성
  recent_5days = np.zeros(len(record_data)) # 최근 5일 경기 타율
  recent_5days_hitter = np.empty(len(record_data), dtype=np.chararray) # type : 문자열 저장을 위해 chararray
  recent_5days_day = np.empty(len(record_data), dtype=np.int32) # 기준이 되는 날짜

  ps = 0
  for h in tqdm.tqdm(hitter_data['batter']): # 90명 타자 한명씩 접근
    hdata = record_data.loc[record_data['name']==h] # 특정 타자 한명에 대해서만 데이터 추출
    for r in hdata['day']: # 추출한 데이터에서 한 행씩 접근
      recent_5days_hitter[ps] = h # 현재 타자
      recent_5days_day[ps] = r  # 현재 기준 날짜
      hrdata = hdata.loc[hdata['day']>=r-recent_day].loc[hdata['day']<=r-1] # 접근한 행의 날짜를 기준으로 5일 경기 
      if len(hrdata)==0: # 최근 5일 경기 없으면 넘어가고
        ps+=1
        continue
      else : # 있으면
        sum_ab = sum(hrdata['ab'])
        if sum_ab==0: # 경기 기록은 존재하지만 타수(ab)가 0
          ps+=1
          continue
        sum_hit = sum(hrdata['hit'])
        recent_5days[ps] = sum_hit/sum_ab
        ps+=1

  result_Dataframe = pd.DataFrame()
  result_Dataframe['recent_5days_hitter'] = recent_5days_hitter
  result_Dataframe['recent_5days_day'] = recent_5days_day
  result_Dataframe['recent_5days'] = recent_5days
  result_Dataframe

  return result_Dataframe

In [8]:
make_recent_days(hitter, data)

100%|██████████| 90/90 [00:08<00:00, 10.53it/s]


,recent_5days_hitter,recent_5days_day,recent_5days
0,전준우,44290,0.000000
1,전준우,44292,0.500000
2,전준우,44293,0.714286
3,전준우,44294,0.636364
4,전준우,44295,0.533333
...,...,...,...
5291,김민하,44367,0.222222
5292,김민하,44369,0.000000
5293,김민하,44370,0.000000
5294,김민하,44371,0.100000
